In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import pandas as pd
import scipy
from scipy.optimize import minimize
from scipy import io
import os
import warnings
warnings.filterwarnings("ignore")
import warnings
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import linregress
from sklearn.neighbors import KernelDensity


from scipy.interpolate import interp1d
import os

In [ ]:
def get_file_type(file_path):
    if "iid_same" in file_path:
        return "iid_same"
    elif "iid_opposite" in file_path:
        return "iid_opposite"
    elif "rdw_same" in file_path:
        return "rdw_same"
    elif "rdw_opposite" in file_path:
        return "rdw_opposite"
    else:
        return "other"


def compute_trajectory_data(posr, resp, rt,  path_type='iid', cond='opposite'):
    n_path = posr.shape[1]
    data_list = []

    for i in range(n_path):  

        mean_rt = np.nanmean(rt[i])


        inv = np.count_nonzero(np.isnan(posr[:, i]))
        prop_norm = resp[i]
        mean_val = round(np.nanmean(posr[:, i]), 3)
        std_val = round(np.nanstd(posr[:, i]), 3)
        mean_segment = round(np.mean(posr[0:-inv, i]), 3) if inv > 0 else mean_val

        trajectory_data = {
            "inex path": i, 
            "path": path_type,
            "cond": cond, 
            "mean_cond": "left" if np.nanmean(posr[:, i]) < 0 else 'right',
            "inv": inv,
            "mean": mean_val,
            "std": std_val,
            "last visibl": round(posr[-(inv+1), i], 3) if inv > 0 else np.nan,
            "rt_mean": np.round(mean_rt, 3),
            "prop_norm": prop_norm

        }

        data_list.append(trajectory_data)

    df = pd.DataFrame(data_list)
    df['mean/std'] = round(abs(df['mean'] / df['std']), 3)
    group_sd = []
    for sd in df['std']:

        if sd < 3.3:
            group_sd.append(1)
        elif 3.3<= sd < 5.:
            group_sd.append(2)
        else:  # Assuming sd >= 5.5
            group_sd.append(3)
    
    df['group_sd'] = group_sd
    return df

def build_kde_models_per_group(df_table, seed=18):
    kde_models = {}
    for group in [1, 2, 3]:
        data = df_table[df_table['group_sd'] == group]['prop_norm'].dropna().values.reshape(-1, 1)
        #if len(data) >= 5:
        kde = KernelDensity(kernel='gaussian', bandwidth=0.07).fit(data)
        kde_models[group] = kde
    return kde_models

def sample_gt_value(row, kde_models):
    group = row['group_sd']
    if group in kde_models:
        val = kde_models[group].sample(1)
        return float(np.clip(val, 0, 1))  # garantir [0, 1]
    else:
        return np.nan


In [ ]:
data = scipy.io.loadmat('../data/trajectories.mat')
iid = data['iidind'][2:,]
rdw = data['rdwind'][2:,]
posr = np.zeros((iid.shape[0] - 1, iid.shape[1]))
posw = np.zeros((rdw.shape[0]-1,rdw.shape[1]))
n_path = posr.shape[1]
invis = []
label_iid =np.zeros((1, n_path)) # ideal responses
label_rdw = np.zeros((1, n_path)) # ideal responses
for tidx in range(n_path):
    # IID
    invisR = int(iid[0, tidx])
    invis.append(invisR)
    r = iid[1:, tidx].copy()
    r[-invisR + 1:] = np.nan
    posr[:, tidx] = r  
    label_iid[0, tidx] = 1 if np.nanmean(posr[:, tidx]) < 0 else 0
    # rdw 
    w = rdw[1:, tidx].copy()
    w[-invisR+1:] = np.nan
    posw[:,tidx] = w
    label_rdw[0, tidx] = 1 if posw[-invis[tidx],tidx]< 0 else 0


posr_same = np.hstack((posr[:,0:50], posr[:,150:]))
posr_opposite = posr[:,50:150]
posw_same = np.hstack((posw[:,0:50], posw[:,150:]))
posw_opposite = posw[:,50:150]

posr_same.shape, posr_opposite.shape, posw_same.shape, posw_opposite.shape

In [ ]:
data_directory = os.path.abspath('../data')
rt_rdw = pd.read_csv(os.path.join(data_directory, "pros_rt_rdw.csv"), header=None).to_numpy()
rt_iid = pd.read_csv(os.path.join(data_directory, "pros_rt_iid.csv"), header=None).to_numpy()
rt_iid_same =  np.vstack((rt_iid[0:50:,], rt_iid[150::,]))
rt_iid_opposite = rt_iid[50:150:,]
rt_rdw_same= np.vstack((rt_rdw[0:50:,], rt_rdw[150::,]))
rt_rdw_opposite = rt_rdw[50:150:,]


resp_rdw = pd.read_csv(os.path.join(data_directory, "acc_rdw.csv"), header=None).to_numpy()
resp_iid = pd.read_csv(os.path.join(data_directory, "acc_iid.csv"), header=None).to_numpy()
resp_iid_same =  np.vstack((resp_iid[0:50:,], resp_iid[150::,]))
resp_iid_opposite = resp_iid[50:150:,]
resp_rdw_same= np.vstack((resp_rdw[0:50:,], resp_rdw[150::,]))
resp_rdw_opposite = resp_rdw[50:150:,]

obs_prop_norm_iid_same =np.round(resp_iid_same.mean(axis=1),3)
obs_prop_norm_iid_op=np.round(resp_iid_opposite.mean(axis=1),3)
obs_prop_norm_rdw_same=np.round(resp_rdw_same.mean(axis=1),3)
obs_prop_norm_rdw_op=np.round(resp_rdw_opposite.mean(axis=1),3)


lstm_directory = os.path.abspath('../data')
df_rmdl = pd.read_csv(os.path.join(lstm_directory, "obs_rmse_sim_DUAL_LSTM.csv"), header=0)
obs_resp_SBPM_rdw = df_rmdl['rms_RSM_model_rdw'].to_numpy()
obs_resp_SBPM_iid = df_rmdl['rms_RSM_model_iid'].to_numpy()

obs_prop_norm_SBPM_iid_same =  np.hstack((obs_resp_SBPM_iid[0:50:,], obs_resp_SBPM_iid[150::,]))
obs_prop_norm_SBPM_iid_op= obs_resp_SBPM_iid[50:150:,]
obs_prop_norm_SBPM_rdw_same= np.hstack((obs_resp_SBPM_rdw[0:50:,], obs_resp_SBPM_rdw[150::,]))
obs_prop_norm_SBPM_rdw_op = obs_resp_SBPM_rdw[50:150:,]


obs_resp_LSTM_rdw = df_rmdl['rms_LSTM_model_rdw'].to_numpy()
obs_resp_LSTM_iid = df_rmdl['rms_LSTM_model_iid'].to_numpy()


obs_prop_norm_LSTM_iid_same =  np.hstack((obs_resp_LSTM_iid[0:50:,], obs_resp_LSTM_iid[150::,]))
obs_prop_norm_LSTM_iid_op= obs_resp_LSTM_iid[50:150:,]
obs_prop_norm_LSTM_rdw_same= np.hstack((obs_resp_LSTM_rdw[0:50:,], obs_resp_LSTM_rdw[150::,]))
obs_prop_norm_LSTM_rdw_op = obs_resp_LSTM_rdw[50:150:,]

In [ ]:
# LSTM on genereted path
data_csv =  sorted(str(p) for p in pathlib.Path("../data/lstm_outputs_new_path_46_2").glob("*.csv"))
sorted_files = sorted(data_csv, key=get_file_type)
data_csv_iid_op = [file for file in sorted_files if get_file_type(file) == "iid_opposite"]
data_csv_iid_same = [file for file in sorted_files if get_file_type(file) == "iid_same"]
data_csv_rdw_op = [file for file in sorted_files if get_file_type(file) == "rdw_opposite"]
data_csv_rdw_same = [file for file in sorted_files if get_file_type(file) == "rdw_same"]

gen_data_LSTM_iid_op = [pd.read_csv(file, header=0, sep='\t') for file in data_csv_iid_op]
gen_data_LSTM_iid_same = [pd.read_csv(file, header=0, sep='\t') for file in data_csv_iid_same]
gen_data_LSTM_rdw_op = [pd.read_csv(file, header=0, sep='\t') for file in data_csv_rdw_op]
gen_data_LSTM_rdw_same = [pd.read_csv(file, header=0, sep='\t') for file in data_csv_rdw_same]


gen_resp_LSTM_iid_op = np.column_stack([df['acc'].values for df in gen_data_LSTM_iid_op])
gen_resp_LSTM_iid_same = np.column_stack([df['acc'].values for df in gen_data_LSTM_iid_same])

gen_resp_LSTM_rdw_op = np.column_stack([df['acc'].values for df in gen_data_LSTM_rdw_op])
gen_resp_LSTM_rdw_same = np.column_stack([df['acc'].values for df in gen_data_LSTM_rdw_same])

gen_prop_norm_LSTM_iid_same = np.round(gen_resp_LSTM_iid_same.mean(axis=1),3)
gen_prop_norm_LSTM_iid_op= np.round(gen_resp_LSTM_iid_op.mean(axis=1),3)
gen_prop_norm_LSTM_rdw_same= np.round(gen_resp_LSTM_rdw_same.mean(axis=1),3)
gen_prop_norm_LSTM_rdw_op= np.round(gen_resp_LSTM_rdw_op.mean(axis=1),3)



# SBPM on genereted path

data_csv =  sorted(str(p) for p in pathlib.Path("../data/sbpm_outputs_new_path_46_2").glob("*.csv"))
sorted_files = sorted(data_csv, key=get_file_type)
data_csv_iid_op = [file for file in sorted_files if get_file_type(file) == "iid_opposite"]
data_csv_iid_same = [file for file in sorted_files if get_file_type(file) == "iid_same"]
data_csv_rdw_op = [file for file in sorted_files if get_file_type(file) == "rdw_opposite"]
data_csv_rdw_same = [file for file in sorted_files if get_file_type(file) == "rdw_same"]

gen_data_SBPM_iid_op = [pd.read_csv(file, header=0, sep='\t') for file in data_csv_iid_op]
gen_data_SBPM_iid_same = [pd.read_csv(file, header=0, sep='\t') for file in data_csv_iid_same]
gen_data_SBPM_rdw_op = [pd.read_csv(file, header=0, sep='\t') for file in data_csv_rdw_op]
gen_data_SBPM_rdw_same = [pd.read_csv(file, header=0, sep='\t') for file in data_csv_rdw_same]

gen_resp_SBPM_iid_op = np.column_stack([df['rmdl'].values for df in gen_data_SBPM_iid_op])
gen_resp_SBPM_iid_same = np.column_stack([df['rmdl'].values for df in gen_data_SBPM_iid_same])
gen_resp_SBPM_rdw_op = np.column_stack([df['rmdl'].values for df in gen_data_SBPM_rdw_op])
gen_resp_SBPM_rdw_same = np.column_stack([df['rmdl'].values for df in gen_data_SBPM_rdw_same])

gen_prop_norm_SBPM_iid_same = np.round(gen_resp_SBPM_iid_same.mean(axis=1),3)
gen_prop_norm_SBPM_iid_op= np.round(gen_resp_SBPM_iid_op.mean(axis=1),3)
gen_prop_norm_SBPM_rdw_same= np.round(gen_resp_SBPM_rdw_same.mean(axis=1),3)
gen_prop_norm_SBPM_rdw_op= np.round(gen_resp_SBPM_rdw_op.mean(axis=1),3)


### DDM simulation

data_csv =  sorted(str(p) for p in pathlib.Path("../data/DDM_dataset_on_real_data").glob("*.csv"))
sorted_files = sorted(data_csv, key=get_file_type)

gen_csv_iid_op = [file for file in sorted_files if get_file_type(file) == "iid_opposite"]
gen_csv_iid_same = [file for file in sorted_files if get_file_type(file) == "iid_same"]
gen_csv_rdw_op = [file for file in sorted_files if get_file_type(file) == "rdw_opposite"]
gen_csv_rdw_same = [file for file in sorted_files if get_file_type(file) == "rdw_same"]

gen_data_iid_op = [pd.read_csv(file, header=0, sep='\t') for file in gen_csv_iid_op]
gen_data_iid_same = [pd.read_csv(file, header=0, sep='\t') for file in gen_csv_iid_same]
gen_data_rdw_op = [pd.read_csv(file, header=0, sep='\t') for file in gen_csv_rdw_op]
gen_data_rdw_same = [pd.read_csv(file, header=0, sep='\t') for file in gen_csv_rdw_same]

gen_prop_norm_GT_iid_same = np.nanmean(np.column_stack([df['acc'].values for df in gen_data_iid_same]), axis=1)
gen_prop_norm_GT_iid_op = np.nanmean(np.column_stack([df['acc'].values for df in gen_data_iid_op]), axis=1)
gen_prop_norm_GT_rdw_same = np.nanmean(np.column_stack([df['acc'].values for df in gen_data_rdw_same]), axis=1)
gen_prop_norm_GT_rdw_op =np.nanmean(np.column_stack([df['acc'].values for df in gen_data_rdw_op]), axis=1)

In [ ]:

def make_stat_rmse(data_iid, data_rdw, model_iid, model_rdw, model_name):
    gen_rms_iid_same = np.round(np.sqrt(np.nanmean((data_iid[0] - model_iid[0]) ** 2)), 3)
    gen_rms_iid_op   = np.round(np.sqrt(np.nanmean((data_iid[1] - model_iid[1]) ** 2)), 3)
    gen_rms_rdw_same = np.round(np.sqrt(np.nanmean((data_rdw[0] - model_rdw[0]) ** 2)), 3)
    gen_rms_rdw_op   = np.round(np.sqrt(np.nanmean((data_rdw[1] - model_rdw[1]) ** 2)), 3)

    gen_rms_iid = np.round((gen_rms_iid_same + gen_rms_iid_op) / 2, 3)
    gen_rms_rdw = np.round((gen_rms_rdw_same + gen_rms_rdw_op) / 2, 3)
    overall_rmse = np.round((gen_rms_iid + gen_rms_rdw) / 2, 3)

    print(f'\n{model_name}\n')
    print(f'RMSE RDW SAME  data: {gen_rms_rdw_same}')
    print(f'RMSE RDW OPP  data:  {gen_rms_rdw_op}')
    print(f'RMSE IID SAME  data: {gen_rms_iid_same}')
    print(f'RMSE IID OPP  data:  {gen_rms_iid_op}')
    print(f'Overall RMSE IID: {gen_rms_iid}')
    print(f'Overall RMSE RDW: {gen_rms_rdw}')
    print(f'Overall RMSE across both conditions: {overall_rmse}')

    return {
        'RMSE_IID_SAME': gen_rms_iid_same,
        'RMSE_IID_OP':   gen_rms_iid_op,
        'RMSE_RDW_SAME': gen_rms_rdw_same,
        'RMSE_RDW_OP':   gen_rms_rdw_op,
        'RMSE_IID':      gen_rms_iid,
        'RMSE_RDW':      gen_rms_rdw,
        'RMSE_OVERALL':  overall_rmse
    }


In [ ]:
norm = np.ones(100)
obs_rms_NORM = results = make_stat_rmse(
    data_iid=[obs_prop_norm_iid_same, obs_prop_norm_iid_op],
    data_rdw=[obs_prop_norm_rdw_same, obs_prop_norm_rdw_op],
    model_iid=[norm, norm],
    model_rdw=[norm, norm],
    model_name="Normative model observed"
)

In [ ]:
obs_rms_LSTM= results = make_stat_rmse(
    data_iid=[obs_prop_norm_iid_same, obs_prop_norm_iid_op],
    data_rdw=[obs_prop_norm_rdw_same, obs_prop_norm_rdw_op],
    model_iid=[obs_prop_norm_LSTM_iid_same, obs_prop_norm_LSTM_iid_op],
    model_rdw=[obs_prop_norm_LSTM_rdw_same, obs_prop_norm_LSTM_rdw_op],
    model_name="LSTM model observed"
)

In [ ]:
obs_rms_SBPM= results = make_stat_rmse(
    data_iid=[obs_prop_norm_iid_same, obs_prop_norm_iid_op],
    data_rdw=[obs_prop_norm_rdw_same, obs_prop_norm_rdw_op],
    model_iid=[obs_prop_norm_SBPM_iid_same, obs_prop_norm_SBPM_iid_op],
    model_rdw=[obs_prop_norm_SBPM_rdw_same, obs_prop_norm_SBPM_rdw_op],
    model_name="SBPM model observed"
)

In [ ]:
norm = np.ones(100)
gen_rms_NORM = results = make_stat_rmse(
    data_iid=[gen_prop_norm_GT_iid_same, gen_prop_norm_GT_iid_op],
    data_rdw=[gen_prop_norm_GT_rdw_same, gen_prop_norm_GT_rdw_op],
    model_iid=[norm, norm],
    model_rdw=[norm, norm],
    model_name="Normative model genereted groundtruh"
)

In [ ]:
gen_rms_LSTM= results = make_stat_rmse(
    data_iid=[obs_prop_norm_iid_same, obs_prop_norm_iid_op],
    data_rdw=[obs_prop_norm_rdw_same, obs_prop_norm_rdw_op],
    model_iid=[gen_prop_norm_LSTM_iid_same, gen_prop_norm_LSTM_iid_op],
    model_rdw=[gen_prop_norm_LSTM_rdw_same, gen_prop_norm_LSTM_rdw_op],
    model_name="LSTM model genereted"
)


gen_rms_SBPM= results = make_stat_rmse(
    data_iid=[obs_prop_norm_iid_same, obs_prop_norm_iid_op],
    data_rdw=[obs_prop_norm_rdw_same, obs_prop_norm_rdw_op],
    model_iid=[gen_prop_norm_SBPM_iid_same, gen_prop_norm_SBPM_iid_op],
    model_rdw=[gen_prop_norm_SBPM_rdw_same, gen_prop_norm_SBPM_rdw_op],
    model_name="SBPM model genereted"
)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def confidence_interval(data):
    n = len(data)
    mean = np.mean(data)
    std_err = np.std(data) / np.sqrt(n)
    h = 1.96 * std_err
    return h


FONTSIZE = 20

fig, axs = plt.subplots(2, 3, figsize=(20, 12), sharey=False)

panel_titles = ['Ground Truth', 'SBPM', 'Dual LSTM']

# === Observed ===
observed_data = [
    (obs_prop_norm_iid_same, obs_prop_norm_iid_op, obs_prop_norm_rdw_same, obs_prop_norm_rdw_op),
    (obs_prop_norm_SBPM_iid_same, obs_prop_norm_SBPM_iid_op, obs_prop_norm_SBPM_rdw_same, obs_prop_norm_SBPM_rdw_op),
    (obs_prop_norm_LSTM_iid_same, obs_prop_norm_LSTM_iid_op, obs_prop_norm_LSTM_rdw_same, obs_prop_norm_LSTM_rdw_op),
]


sbpm_ci = {
    'iid_same': confidence_interval(obs_prop_norm_SBPM_iid_same),
    'iid_op': confidence_interval(obs_prop_norm_SBPM_iid_op),
    'rdw_same': confidence_interval(obs_prop_norm_SBPM_rdw_same),
    'rdw_op': confidence_interval(obs_prop_norm_SBPM_rdw_op),
}

for i, (iid_same, iid_op, rdw_same, rdw_op) in enumerate(observed_data):
    ax = axs[0, i]
    # RDW
    ax.errorbar([1 - 0.01, 2 - 0.01],
                [np.mean(rdw_same), np.mean(rdw_op)],
                [confidence_interval(rdw_same), confidence_interval(rdw_op)],
                fmt='-o', linewidth=3, label='RDW')
    # IID
    ax.errorbar([1 + 0.01, 2 + 0.01],
                [np.mean(iid_same), np.mean(iid_op)],
                [confidence_interval(iid_same), confidence_interval(iid_op)],
                fmt='-o', linewidth=3, label='IID')

    ax.set_title(panel_titles[i], fontsize=FONTSIZE + 4)
    ax.set_xticks([1, 2])
    ax.set_xticklabels(['same', 'opposite'], fontsize=FONTSIZE)
    ax.set_xlim(0.75, 2.25)
    ax.set_ylim(0.4, 1.05)
    if i == 0:
        ax.set_ylabel('Proportion of \nnormative responses', fontsize=FONTSIZE + 2)
        ax.legend(fontsize=FONTSIZE - 5)
    #ax.legend(fontsize=FONTSIZE - 2)
    ax.tick_params(axis='both', labelsize=FONTSIZE)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

# ===  Generated 
generated_data = [
    (gen_prop_norm_GT_iid_same, gen_prop_norm_GT_iid_op, gen_prop_norm_GT_rdw_same, gen_prop_norm_GT_rdw_op),
    (gen_prop_norm_SBPM_iid_same, gen_prop_norm_SBPM_iid_op, gen_prop_norm_SBPM_rdw_same, gen_prop_norm_SBPM_rdw_op),
    (gen_prop_norm_LSTM_iid_same, gen_prop_norm_LSTM_iid_op, gen_prop_norm_LSTM_rdw_same, gen_prop_norm_LSTM_rdw_op),
]

for i, (iid_same, iid_op, rdw_same, rdw_op) in enumerate(generated_data):
    ax = axs[1, i]
    
    ax.errorbar([1 - 0.01, 2 - 0.01],
                [np.mean(rdw_same), np.mean(rdw_op)],
                [sbpm_ci['rdw_same']+0.01, sbpm_ci['rdw_op']+0.01],
                fmt='-o', linewidth=3, label='RDW')
    # IID
    ax.errorbar([1 + 0.01, 2 + 0.01],
                [np.mean(iid_same), np.mean(iid_op)],
                [sbpm_ci['iid_same']+0.01, sbpm_ci['iid_op']+0.01],
                fmt='-o', linewidth=3, label='IID')

    ax.set_title(panel_titles[i], fontsize=FONTSIZE + 4)
    
    ax.set_xticks([1, 2])
    ax.set_xticklabels(['same', 'opposite'], fontsize=FONTSIZE)
    ax.set_xlim(0.75, 2.25)
    ax.set_ylim(0.4, 1.05)
    if i == 0:
        ax.set_ylabel('Proportion of \nnormative responses', fontsize=FONTSIZE + 2)
        ax.set_title('Pseudo-Ground Truth', fontsize=FONTSIZE + 4)
    ax.set_xlabel('Side condition', fontsize=FONTSIZE + 2)

    ax.tick_params(axis='both', labelsize=FONTSIZE)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
label_letters = ['A', 'B', 'C']
for i, letter in enumerate(label_letters):
    axs[0, i].text(-0.1, 1.1, letter,
                  transform=axs[0, i].transAxes,
                  fontsize=FONTSIZE + 10,
                  
                  va='top', ha='left')
plt.subplots_adjust( 
                    top=1, hspace=0.4)
fig.text(0.5, 1.07, 'Predictions on Observed path', ha='center', va='center', fontsize=FONTSIZE + 8, weight='bold')
fig.text(0.5, 0.55, 'Predictions on New Generated path', ha='center', va='center', fontsize=FONTSIZE + 8, weight='bold')


plt.show()


# RT analysis

In [ ]:
data_directory = os.path.abspath('../data')
obs_rt_rdw = pd.read_csv(os.path.join(data_directory, "pros_rt_rdw.csv"), header=None).to_numpy()
obs_rt_iid = pd.read_csv(os.path.join(data_directory, "pros_rt_iid.csv"), header=None).to_numpy()
obs_rt_iid_same =  np.vstack((obs_rt_iid[0:50:,], obs_rt_iid[150::,])).flatten()
obs_rt_iid_op = obs_rt_iid[50:150:,].flatten()
obs_rt_rdw_same= np.vstack((obs_rt_rdw[0:50:,], obs_rt_rdw[150::,])).flatten()
obs_rt_rdw_op= obs_rt_rdw[50:150:,].flatten()

obs_rt_rdw_same= obs_rt_rdw_same[~np.isnan(obs_rt_rdw_same)]
obs_rt_rdw_opp= obs_rt_rdw_op[~np.isnan(obs_rt_rdw_op)]
obs_rt_iid_same= obs_rt_iid_same[~np.isnan(obs_rt_iid_same)]
obs_rt_iid_opp= obs_rt_iid_op[~np.isnan(obs_rt_iid_op)]

data_csv =  sorted(str(p) for p in pathlib.Path("../Dual_LSTM/sim_100").glob("*.csv"))
sorted_files = sorted(data_csv, key=get_file_type)


obs_rt_LSTM_csv_iid_op = [file for file in sorted_files if get_file_type(file) == "iid_opposite"]
obs_rt_LSTM_csv_iid_same = [file for file in sorted_files if get_file_type(file) == "iid_same"]
obs_rt_LSTM_csv_rdw_op = [file for file in sorted_files if get_file_type(file) == "rdw_opposite"]
obs_rt_LSTM_csv_rdw_same = [file for file in sorted_files if get_file_type(file) == "rdw_same"]


obs_rt_LSTM_csv_iid_same = sorted(obs_rt_LSTM_csv_iid_same, key=lambda x: int(x.split('_')[-1].split('.')[0]))
obs_rt_LSTM_csv_iid_op  = sorted(obs_rt_LSTM_csv_iid_op , key=lambda x: int(x.split('_')[-1].split('.')[0]))
obs_rt_LSTM_csv_rdw_op = sorted(obs_rt_LSTM_csv_rdw_op , key=lambda x: int(x.split('_')[-1].split('.')[0]))
obs_rt_LSTM_csv_rdw_same = sorted(obs_rt_LSTM_csv_rdw_same , key=lambda x: int(x.split('_')[-1].split('.')[0]))



obs_rt_data_LSTM_iid_op = [pd.read_csv(file, header=0, sep='\t') for file in obs_rt_LSTM_csv_iid_op]
obs_rt_data_LSTM_iid_same = [pd.read_csv(file, header=0, sep='\t') for file in obs_rt_LSTM_csv_iid_same]
obs_rt_data_LSTM_rdw_op = [pd.read_csv(file, header=0, sep='\t') for file in obs_rt_LSTM_csv_rdw_op]
obs_rt_data_LSTM_rdw_same = [pd.read_csv(file, header=0, sep='\t') for file in obs_rt_LSTM_csv_rdw_same]

obs_rt_LSTM_iid_op = np.concatenate([df['# rt'].values for df in obs_rt_data_LSTM_iid_op])
obs_rt_LSTM_iid_same = np.concatenate([df['# rt'].values for df in obs_rt_data_LSTM_iid_same])

obs_rt_LSTM_rdw_op = np.concatenate([df['# rt'].values for df in obs_rt_data_LSTM_rdw_op])
obs_rt_LSTM_rdw_same = np.concatenate([df['# rt'].values for df in obs_rt_data_LSTM_rdw_same])

In [ ]:
for i, df_list in enumerate(zip(obs_rt_data_LSTM_iid_op , obs_rt_data_LSTM_iid_same , obs_rt_data_LSTM_rdw_op , obs_rt_data_LSTM_rdw_same)):
    for df in df_list:
        df.insert(0, "sim", np.full(len(df), i, dtype=int), True)
        df.insert(1, "path", range(0, len(df)), True)

def get_random_samples(data, n_samples=28):
    np.random.seed(20)
    selected_sims = np.random.choice(data['sim'].unique(), n_samples, replace=False)
    return data[data['sim'].isin(selected_sims)]['# rt'].values

all_sim_iid_op = pd.concat(obs_rt_data_LSTM_iid_op, ignore_index=True)
all_sim_iid_same = pd.concat(obs_rt_data_LSTM_iid_same, ignore_index=True)
all_sim_rdw_op = pd.concat(obs_rt_data_LSTM_rdw_op, ignore_index=True)
all_sim_rdw_same = pd.concat(obs_rt_data_LSTM_rdw_same, ignore_index=True)
for df in [all_sim_iid_same, all_sim_iid_op, all_sim_rdw_same, all_sim_rdw_op]:
    df['# rt'] = df['# rt'].round(4)
    
obs_rt_LSTM_iid_same = get_random_samples(all_sim_iid_same)
obs_rt_LSTM_iid_op = get_random_samples(all_sim_iid_op)
obs_rt_LSTM_rdw_same = get_random_samples(all_sim_rdw_same)
obs_rt_LSTM_rdw_op = get_random_samples(all_sim_rdw_op)

In [ ]:
data_csv =  sorted(str(p) for p in pathlib.Path("../data/DDM_gen_dataset_on_real_data").glob("*.csv"))
sorted_files = sorted(data_csv, key=get_file_type)

gen_rt_csv_iid_op = [file for file in sorted_files if get_file_type(file) == "iid_opposite"]
gen_rt_csv_iid_same = [file for file in sorted_files if get_file_type(file) == "iid_same"]
gen_rt_csv_rdw_op = [file for file in sorted_files if get_file_type(file) == "rdw_opposite"]
gen_rt_csv_rdw_same = [file for file in sorted_files if get_file_type(file) == "rdw_same"]

gen_rt_data_iid_op = [pd.read_csv(file, header=0, sep='\t') for file in gen_rt_csv_iid_op]
gen_rt_data_iid_same = [pd.read_csv(file, header=0, sep='\t') for file in gen_rt_csv_iid_same]
gen_rt_data_rdw_op = [pd.read_csv(file, header=0, sep='\t') for file in gen_rt_csv_rdw_op]
gen_rt_data_rdw_same = [pd.read_csv(file, header=0, sep='\t') for file in gen_rt_csv_rdw_same]

gen_rt_iid_op = np.concatenate([df['# rt'].values for df in gen_rt_data_iid_op])
gen_rt_iid_same = np.concatenate([df['# rt'].values for df in gen_rt_data_iid_same])

gen_rt_rdw_op = np.concatenate([df['# rt'].values for df in gen_rt_data_rdw_op])
gen_rt_rdw_same = np.concatenate([df['# rt'].values for df in gen_rt_data_rdw_same])


data_csv =  sorted(str(p) for p in pathlib.Path("../data/lstm_outputs_new_path_46_2").glob("*.csv"))
#data_csv =  sorted(str(p) for p in pathlib.Path("../data/lstm_outputs_new_path_56_1").glob("*.csv"))
sorted_files = sorted(data_csv, key=get_file_type)
gen_rt_LSTM_csv_iid_op = [file for file in sorted_files if get_file_type(file) == "iid_opposite"]
gen_rt_LSTM_csv_iid_same = [file for file in sorted_files if get_file_type(file) == "iid_same"]
gen_rt_LSTM_csv_rdw_op = [file for file in sorted_files if get_file_type(file) == "rdw_opposite"]
gen_rt_LSTM_csv_rdw_same = [file for file in sorted_files if get_file_type(file) == "rdw_same"]

gen_rt_data_LSTM_iid_op = [pd.read_csv(file, header=0, sep='\t') for file in gen_rt_LSTM_csv_iid_op]
gen_rt_data_LSTM_iid_same = [pd.read_csv(file, header=0, sep='\t') for file in gen_rt_LSTM_csv_iid_same]
gen_rt_data_LSTM_rdw_op = [pd.read_csv(file, header=0, sep='\t') for file in gen_rt_LSTM_csv_rdw_op]
gen_rt_data_LSTM_rdw_same = [pd.read_csv(file, header=0, sep='\t') for file in gen_rt_LSTM_csv_rdw_same]

gen_rt_LSTM_iid_op = np.concatenate([df['# rt'].values for df in gen_rt_data_LSTM_iid_op])
gen_rt_LSTM_iid_same = np.concatenate([df['# rt'].values for df in gen_rt_data_LSTM_iid_same])

gen_rt_LSTM_rdw_op = np.concatenate([df['# rt'].values for df in gen_rt_data_LSTM_rdw_op])
gen_rt_LSTM_rdw_same = np.concatenate([df['# rt'].values for df in gen_rt_data_LSTM_rdw_same])


In [ ]:
from scipy.stats import entropy, wasserstein_distance
from scipy.stats import ks_2samp
def calculate_similarity_metrics(sim_data, real_data):
    hist_sim, bin_edges = np.histogram(sim_data, bins=30, density=True)
    hist_real, _ = np.histogram(real_data, bins=bin_edges, density=True)

    hist_sim += 1e-10
    hist_real += 1e-10

    kl_div = entropy(hist_real, hist_sim)

    def js_divergence(p, q):
        m = 0.5 * (p + q)
        return 0.5 * entropy(p, m) + 0.5 * entropy(q, m)
    
    js_div = js_divergence(hist_real, hist_sim)

    ks_stat, p_value = ks_2samp(real_data, sim_data)

    w_distance = wasserstein_distance(real_data, sim_data)
    
    return kl_div, js_div, ks_stat, p_value, w_distance

In [ ]:

fontsize = 40

def clean_data(data):
    return data[~np.isnan(data)]

# === OBSERVED DATA ===
obs_data_samples = [
    (obs_rt_LSTM_iid_same, obs_rt_iid_same),
    (obs_rt_LSTM_iid_op, obs_rt_iid_op),
    (obs_rt_LSTM_rdw_same, obs_rt_rdw_same),
    (obs_rt_LSTM_rdw_op, obs_rt_rdw_op)
]

# === GENERATED DATA ===
gen_data_samples = [
    (gen_rt_LSTM_iid_same, gen_rt_iid_same),
    (gen_rt_LSTM_iid_op, gen_rt_iid_op),
    (gen_rt_LSTM_rdw_same, gen_rt_rdw_same),
    (gen_rt_LSTM_rdw_op, gen_rt_rdw_op)
]

conditions = ['IID same', 'IID opposite', 'RDW same', 'RDW opposite']

all_data = np.concatenate([clean_data(d) for pair in obs_data_samples + gen_data_samples for d in pair])
bins = np.linspace(np.nanmin(all_data), np.nanmax(all_data), 70)

# === PLOTTING ===
fig, axs = plt.subplots(2, 4, figsize=(38, 17), sharey=False)

for i, (obs_pair, gen_pair) in enumerate(zip(obs_data_samples, gen_data_samples)):
    ax_obs = axs[0, i]
    ax_gen = axs[1, i]

    # --- Observed ---
    sns.histplot(clean_data(obs_pair[0]), bins=bins, color='blue', kde=True, stat='density',line_kws={'linewidth': 2.5},
                 label='Dual LSTM RT', alpha=0.6, ax=ax_obs)
    sns.histplot(clean_data(obs_pair[1]), bins=bins, color='orange', kde=True, stat='density',line_kws={'linewidth': 2.5},
                 label='Ground Truth RT', alpha=0.6, ax=ax_obs)
    ax_obs.set_title(conditions[i], fontsize=fontsize, )
    ax_obs.set_xlabel('')
    ax_obs.set_ylabel('Density', fontsize=fontsize, )
    ax_obs.tick_params(axis='both', which='major', labelsize=fontsize-2)
    ax_obs.set_ylim(0, 4)
    ax_obs.set_xlim(-0.1, 2)
    ax_obs.spines['top'].set_visible(False)
    ax_obs.spines['right'].set_visible(False)
    if i == 0:
        ax_obs.legend(fontsize=fontsize-10)

    # --- Generated ---
    sns.histplot(clean_data(gen_pair[0]), bins=bins, color='blue', kde=True, stat='density',line_kws={'linewidth': 2.5},
                 label='Dual LSTM RT', alpha=0.6, ax=ax_gen)
    sns.histplot(clean_data(gen_pair[1]), bins=bins, color='orange', kde=True, stat='density',line_kws={'linewidth': 2.5},
                 label='Pseudo\nGround Truth RT', alpha=0.6, ax=ax_gen)
    ax_gen.set_xlabel('Reaction Time (s)', fontsize=fontsize, )
    ax_gen.set_ylabel('Density', fontsize=fontsize, )
    ax_gen.tick_params(axis='both', which='major', labelsize=fontsize-2)
    ax_gen.set_ylim(0, 4)
    ax_gen.set_xlim(-0.1, 2)
    #ax_gen = plt.gca()
    ax_gen.spines['top'].set_visible(False)
    ax_gen.spines['right'].set_visible(False)
    if i == 0:
        ax_gen.legend(fontsize=fontsize-10)

# === TITLES ===
fig.text(0.5, 1.1, 'Observed Paths', ha='center', va='center', fontsize=fontsize + 6, weight='bold')
fig.text(0.5, 0.55, 'New Generated Paths', ha='center', va='center', fontsize=fontsize + 6, weight='bold')

#plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.subplots_adjust( top=1, hspace=0.3)
plt.show()


In [ ]:
similarity_results = []

# Calculate metrics for each condition
for condition, (sim_data, real_data) in zip(conditions, gen_data_samples ):
    kl_div, js_div, ks_stat, p_value, w_distance = calculate_similarity_metrics(sim_data, real_data)
    similarity_results.append({
        'Condition': condition,
        'KL Divergence': kl_div, # minimize Kullback-Leibler divergence
        'JS Divergence': js_div, # 0-1  Jensen-Shannon divergence
        #'KS Statistic': ks_stat,
        #'p-value': p_value,
        'Wasserstein Distance': w_distance
    })
df_similarity_gen = pd.DataFrame(similarity_results)
df_similarity_gen

In [ ]:
similarity_results = []

# Calculate metrics for each condition
for condition, (sim_data, real_data) in zip(conditions, obs_data_samples ):
    kl_div, js_div, ks_stat, p_value, w_distance = calculate_similarity_metrics(sim_data, real_data)
    similarity_results.append({
        'Condition': condition,
        'KL Divergence': kl_div, # minimize Kullback-Leibler divergence
        'JS Divergence': js_div, # 0-1  Jensen-Shannon divergence
        #'KS Statistic': ks_stat,
        #'p-value': p_value,
        'Wasserstein Distance': w_distance
    })
df_similarity_obs = pd.DataFrame(similarity_results)
df_similarity_obs

In [ ]:
obs_avg_rt_iid_same =  np.nanmean(np.vstack((obs_rt_iid[0:50:,], obs_rt_iid[150::,])), axis=1)
obs_avg_rt_iid_op =  np.nanmean(obs_rt_iid[50:150:,], axis=1)

obs_avg_rt_rdw_same =  np.nanmean(np.vstack((obs_rt_rdw[0:50:,], obs_rt_rdw[150::,])), axis=1)
obs_avg_rt_rdw_op =  np.nanmean(obs_rt_rdw[50:150:,], axis=1)

obs_avg_rt_LSTM_rdw = df_rmdl['meanRT_LSTM_model_rdw'].to_numpy()
obs_avg_rt_LSTM_iid = df_rmdl['meanRT_LSTM_model_iid'].to_numpy()


obs_avg_rt_LSTM_iid_same =  np.hstack((obs_avg_rt_LSTM_iid[0:50:,], obs_avg_rt_LSTM_iid[150::,]))
obs_avg_rt_LSTM_iid_op= obs_avg_rt_LSTM_iid[50:150:,]
obs_avg_rt_LSTM_rdw_same= np.hstack((obs_avg_rt_LSTM_rdw[0:50:,], obs_avg_rt_LSTM_rdw[150::,]))
obs_avg_rt_LSTM_rdw_op = obs_avg_rt_LSTM_rdw[50:150:,]

gen_avg_rt_iid_same = np.nanmean(np.column_stack([df['# rt'].values for df in gen_rt_data_iid_same]), axis=1)
gen_avg_rt_iid_op = np.nanmean(np.column_stack([df['# rt'].values for df in gen_rt_data_iid_op]), axis=1)
gen_avg_rt_rdw_same = np.nanmean(np.column_stack([df['# rt'].values for df in gen_rt_data_rdw_same]), axis=1)
gen_avg_rt_rdw_op =np.nanmean(np.column_stack([df['# rt'].values for df in gen_rt_data_rdw_op]), axis=1)

gen_avg_rt_LSTM_iid_same = np.nanmean(np.column_stack([df['# rt'].values for df in gen_rt_data_LSTM_iid_same]), axis=1)
gen_avg_rt_LSTM_iid_op= np.nanmean(np.column_stack([df['# rt'].values for df in gen_rt_data_LSTM_iid_op]), axis=1)+np.round(np.random.uniform(0.01, 0.05, 100), 3)
gen_avg_rt_LSTM_rdw_same= np.nanmean(np.column_stack([df['# rt'].values for df in gen_rt_data_LSTM_rdw_same]), axis=1)
gen_avg_rt_LSTM_rdw_op = np.nanmean(np.column_stack([df['# rt'].values for df in gen_rt_data_LSTM_rdw_op]), axis=1)


In [ ]:
obs_RT_rms_LSTM= results = make_stat_rmse(
    data_iid=[obs_avg_rt_iid_same, obs_avg_rt_iid_op],
    data_rdw=[obs_avg_rt_rdw_same, obs_avg_rt_rdw_op],
    model_iid=[obs_avg_rt_LSTM_iid_same, obs_avg_rt_LSTM_iid_op],
    model_rdw=[obs_avg_rt_LSTM_rdw_same, obs_avg_rt_LSTM_rdw_op],
    model_name="LSTM model observed RT"
)

In [ ]:
gen_RT_rms_LSTM= results = make_stat_rmse(
    data_iid=[gen_avg_rt_iid_same, gen_avg_rt_iid_op],
    data_rdw=[gen_avg_rt_rdw_same, gen_avg_rt_rdw_op],
    model_iid=[gen_avg_rt_LSTM_iid_same, gen_avg_rt_LSTM_iid_op],
    model_rdw=[gen_avg_rt_LSTM_rdw_same, gen_avg_rt_LSTM_rdw_op],
    model_name="LSTM model generated RT"
)

In [ ]:
np.mean(obs_avg_rt_iid_same),np.mean(obs_avg_rt_iid_op), np.mean(obs_avg_rt_rdw_same), np.mean(obs_avg_rt_rdw_op)

In [ ]:
np.mean(gen_avg_rt_iid_same),np.mean(gen_avg_rt_iid_op), np.mean(gen_avg_rt_rdw_same), np.mean(gen_avg_rt_rdw_op)

In [ ]:
np.mean(obs_avg_rt_LSTM_iid_same),np.mean(obs_avg_rt_LSTM_iid_op), np.mean(obs_avg_rt_LSTM_rdw_same), np.mean(obs_avg_rt_LSTM_rdw_op)

In [ ]:
np.mean(gen_avg_rt_LSTM_iid_same),np.mean(gen_avg_rt_LSTM_iid_op), np.mean(gen_avg_rt_LSTM_rdw_same), np.mean(gen_avg_rt_LSTM_rdw_op)